## Импорт нужных модулей

In [1]:
from pymystem3 import Mystem
m = Mystem()
import json
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import nltk
from nltk.tokenize import word_tokenize
import jsonlines
import collections as c
import pandas as pd

## Пункт 2
Обработать книгу с майстем

In [2]:
with open ('vody.txt', encoding = 'utf-8') as f:
    big_text = f.read()
%time lemmas = m.lemmatize(big_text)
with open('data.json', 'a', encoding='utf-8') as j:
    json.dump(lemmas, j, ensure_ascii = False)

Wall time: 10.3 s


## Пункт 3
Обработать книгу с pymorphy

In [4]:
%time tok = word_tokenize(big_text)
for w in tok:
    p = morph.parse(w)[0]
    #my_tag = p.tag.POS
    #print(my_tag)
    info = {'lemma' : p.normal_form, 'word': p.word, 'pos': p.tag.POS}
    #print(info)
    with jsonlines.open('output.jsonl', mode='a') as writer:
        writer.write(info)

Wall time: 319 ms


In [5]:
%time tok = word_tokenize(big_text)
lemmas = []
for w in tok:
    p = morph.parse(w)[0]
    lemmas.append(list(p))

Wall time: 292 ms


## Пункт 4
Какую долю слов составляет каждая часть речи?

In [2]:
pos = c.defaultdict(list)
summ = 0
my_data = {}
for w in tok:
    p = morph.parse(w)[0]
    part = p.tag.POS
    pos[part].append(w)
for p in pos.keys():
    summ += len(pos[p])
for p in pos.keys():
    perc = (len(pos[p])/summ)*100
    print(p, ' составляет ', perc, '%', sep = '')
    my_data[p] = perc
df = pd.DataFrame(my_data)
df

NameError: name 'tok' is not defined

Найдите топ-20 (по частотности) глаголов и наречий

In [7]:
verb = pos['VERB']
count_v = c.Counter(verb)
print(count_v.most_common(20))
adverb = pos['ADVB']
count_a = c.Counter(adverb)
print(count_a.most_common(20))

[('было', 118), ('был', 79), ('будет', 42), ('мог', 40), ('была', 39), ('может', 37), ('спросил', 29), ('мой', 26), ('заметил', 25), ('воскликнул', 25), ('сказала', 24), ('промолвил', 22), ('подумал', 20), ('буду', 20), ('хотел', 19), ('взял', 19), ('спросила', 19), ('отвечал', 19), ('промолвила', 18), ('знал', 17)]
[('уже', 108), ('только', 92), ('очень', 71), ('теперь', 56), ('опять', 52), ('несколько', 49), ('вдруг', 45), ('раз', 44), ('там', 42), ('наконец', 40), ('тотчас', 38), ('потом', 30), ('всего', 29), ('почти', 28), ('никогда', 28), ('тут', 28), ('назад', 27), ('где', 26), ('слегка', 26), ('вперед', 25)]


## Пункт 5
топ-25 биграмм и триграмм

In [9]:
words = [w.lower() for w in tok if w.isalpha()]
wordsbigrm = nltk.bigrams(words)
bg = list(wordsbigrm)
cb = c.Counter(bg)
wordstrigrm = nltk.trigrams(words)
tg = list(wordstrigrm)
ct = c.Counter(tg)
print(cb.most_common(25), ct.most_common(25))

[(('марья', 'николаевна'), 118), (('фрау', 'леноре'), 78), (('что', 'он'), 67), (('и', 'не'), 52), (('что', 'я'), 44), (('я', 'не'), 42), (('на', 'него'), 42), (('с', 'ним'), 42), (('он', 'не'), 41), (('к', 'нему'), 31), (('что', 'вы'), 30), (('и', 'он'), 28), (('и', 'даже'), 27), (('в', 'нем'), 25), (('у', 'меня'), 25), (('как', 'бы'), 24), (('и', 'с'), 24), (('не', 'мог'), 22), (('так', 'и'), 22), (('и', 'в'), 21), (('что', 'она'), 21), (('с', 'вами'), 21), (('я', 'вас'), 20), (('то', 'что'), 20), (('ничего', 'не'), 19)] [(('в', 'первый', 'раз'), 10), (('что', 'он', 'не'), 8), (('к', 'нему', 'и'), 8), (('на', 'него', 'и'), 8), (('марья', 'николаевна', 'и'), 8), (('к', 'санину', 'и'), 7), (('с', 'ним', 'в'), 6), (('на', 'следующий', 'день'), 6), (('в', 'комнату', 'и'), 6), (('то', 'же', 'время'), 6), (('а', 'марья', 'николаевна'), 6), (('со', 'всех', 'сторон'), 5), (('в', 'то', 'же'), 5), (('и', 'как', 'бы'), 5), (('по', 'крайней', 'мере'), 5), (('с', 'своей', 'стороны'), 5), (('но', 